In [1]:
import json
import pandas as pd

In [2]:
collections = pd.read_csv("data_pp/collections.csv")
sentences = pd.read_csv("data_pp/sentences_lang.csv")
sentences_annotated_new = pd.read_csv("data_pp/sentences_annotated_new.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'data_pp/collections.csv'

In [15]:
len(collections)

4924

### Fix some values

In [3]:
collections.loc[collections['project_country_title'] == "Syria - Division Labels,Syria for IMMAP/DFS", "project_country_title"] = "Syria for IMMAP/DFS"
collections.loc[collections['project_title'] == "IMMAP/DFS Syria (IRRELEVANT) [REMOVE ME]", "project_title"] = "IMMAP/DFS Syria"
collections.loc[collections['project_country_title'] == "IMMAP RDC", "project_country_title"] = "IMMAP DRC"
collections.loc[collections["project_country_code"]=="SYRDL,SYR", "project_country_code"] = "SYR"

In [4]:
sentences.loc[sentences['project_country_title'] == "Syria - Division Labels,Syria for IMMAP/DFS", "project_country_title"] = "Syria for IMMAP/DFS"
sentences.loc[sentences['project_title'] == "IMMAP/DFS Syria (IRRELEVANT) [REMOVE ME]", "project_title"] = "IMMAP/DFS Syria"
sentences.loc[sentences['project_country_title'] == "IMMAP RDC", "project_country_title"] = "IMMAP DRC"
sentences.loc[sentences["project_country_code"]=="SYRDL,SYR", "project_country_code"] = "SYR"

In [5]:
sentences_annotated_new.loc[sentences_annotated_new['project_country_title'] == "Syria - Division Labels,Syria for IMMAP/DFS", "project_country_title"] = "Syria for IMMAP/DFS"
sentences_annotated_new.loc[sentences_annotated_new['project_title'] == "IMMAP/DFS Syria (IRRELEVANT) [REMOVE ME]", "project_title"] = "IMMAP/DFS Syria"
sentences_annotated_new.loc[sentences_annotated_new['project_country_title'] == "IMMAP RDC", "project_country_title"] = "IMMAP DRC"

### Prepare The Documents

In [6]:
collections.columns

Index(['project_id', 'project_title', 'project_country_code',
       'project_country_title', 'lead_id', 'url', 'simplified_text'],
      dtype='object')

In [7]:
collections.loc[sentences['project_country_title'] == "IMMAP/DFS Syria (IRRELEVANT) [REMOVE ME]", "project_country_title"]

Series([], Name: project_country_title, dtype: object)

In [8]:
collections["project_title"].value_counts().index.tolist()

['IMMAP/DFS Bangladesh',
 'IMMAP/DFS Syria',
 'IMMAP/DFS Nigeria',
 'IMMAP/DFS Burkina Faso',
 'IMMAP/DFS Colombia',
 'IMMAP/DFS RDC']

In [9]:
collections["project_country_code"].value_counts().index.tolist()

['BGD', 'SYR', 'NGA', 'BFA', 'COL', 'COD']

In [10]:
# documents --> project_title, country_code, lang_code, doc_id, doc_text, doc_url
documents = collections[["project_title", "project_country_code", "lead_id", "simplified_text", "url"]]
documents.columns = ["project_name", "country_code", "doc_id", "doc_text", "doc_url"]
##
documents.to_csv("final_before_split/immap_dfs_documents.csv", index=None)

### Prepare The Sentences

In [11]:
# annotated_sentences --> doc_id, sentence_id, is_relevant, sector_ids
annotations = sentences_annotated_new[['lead_id', 'excerpt_id', 'is_selected',
       'label_sectors', 'label_dimensions', 'unique_id']]
##
# doc_id, sentence_id, is_relevant, sector_id --> annotated_sentences
annotated_sentences = pd.merge(sentences, annotations, how="inner", on=['lead_id', 'excerpt_id'])
#
annotated_sentences = annotated_sentences[['project_id', 'project_title', 'project_country_code',
       'project_country_title', 'lead_id', 'excerpt_id', 'excerpt_text',
       'excerpt_text_pp', 'lang', 'is_selected',
       'label_sectors', 'label_dimensions', 'unique_id_y']]
#
annotated_sentences.columns = ['project_id', 'project_title', 'project_country_code',
       'project_country_title', 'doc_id', 'sentence_id', 'sentence_text',
       'sentence_text_simplified', 'lang_code', 'is_relevant', 'sector_ids', 'label_dimensions',
       'unique_id']
annotated_sentences = annotated_sentences[~annotated_sentences["lang_code"].eq("ar")]
annotated_sentences = annotated_sentences[["doc_id", "sentence_id", "sentence_text", "lang_code", "is_relevant", "sector_ids"]]
##
all_labels = annotated_sentences["sector_ids"].unique().tolist()[1:]
unique_labels = set()
for label in all_labels:
    labels = label.split(", ")
    unique_labels.update(labels)
label_to_id = {label:i for i, label in enumerate(sorted(unique_labels))}
label_to_id
##
def convert_label_to_id(label_array):
    if pd.isna(label_array):
        return []
#     if not isinstance(label_array, str):
#         return label_array
    labels = label_array.split(", ")
    return [label_to_id[label] for label in sorted(labels)]
##
annotated_sentences["sector_ids"] = annotated_sentences["sector_ids"].map(convert_label_to_id)
##
annotated_sentences["is_relevant"] = annotated_sentences["is_relevant"].map({True:1, False:0})
##


In [12]:
annotated_sentences["sentence_text"].duplicated().sum()

0

In [13]:
annotated_sentences.to_csv("final_before_split/immap_dfs_sentences.csv", index=None)

In [14]:
annotated_sentences

,doc_id,sentence_id,sentence_text,lang_code,is_relevant,sector_ids
0,51795,0,Por cuenta de las declaraciones de la alcaldes...,es,1,[]
1,51795,1,Para analizar esta situaci n y las consecuenci...,es,0,[]
2,51795,2,M s all de la errada relaci n de causalidad en...,es,0,[]
3,51795,3,"Aqu falta invertir recursos, tiempo, equipos d...",es,0,[]
4,51795,4,Hay que dejar de hablar de la seguridad de Bog...,es,0,[]
...,...,...,...,...,...,...
355710,34512,121,"However, humanitarian exemptions are not alway...",en,0,[]
355711,34512,122,"In Iraq, there are reports of exemptions from ...",en,0,[]
355712,34512,123,The ACAPS team will keep monitoring the impact...,en,0,[]
355713,34512,124,You can find an overview of all ACAPS resource...,en,0,[]
